In [33]:
from IPython.display import display, Audio
import ipywidgets as widgets

print("=" * 70)
print("🎤 AUDIO EMOTION PREDICTION TEST")
print("=" * 70)
print("\n📤 Upload an audio file (WAV, MP3, M4A, OGG)...")

# زرار الرفع
upload = widgets.FileUpload(
    accept='. wav,.mp3,. m4a,.ogg,. flac',
    multiple=False,
    description='Upload Audio'
)
display(upload)

print("\n⬆️ Click the button above to upload your audio file")

🎤 AUDIO EMOTION PREDICTION TEST

📤 Upload an audio file (WAV, MP3, M4A, OGG)...


FileUpload(value=(), accept='. wav,.mp3,. m4a,.ogg,. flac', description='Upload Audio')


⬆️ Click the button above to upload your audio file


In [36]:
import numpy as np
import librosa
import tensorflow as tf
from IPython.display import Audio, display
import os

model_path = r'D:\Downloads\DL2\results\best_audio_model.h5'

labels_path = r'D:\Downloads\DL2\results\label_encoder_classes. npy.npy' 

print(" Loading model...")
try:
    if not os.path.exists(labels_path):
        print(f" File not found at: {labels_path}")
        print("تأكد من أنك قمت بنسخ المسار واسم الملف بالحرف كما يظهر في الصورة")
    
    model = tf.keras.models.load_model(model_path, compile=False)
    labels = np.load(labels_path, allow_pickle=True)
    print(" Model loaded successfully!")
    print(f" Emotions: {list(labels)}")
except Exception as e:
    print(f" Error loading model: {e}")
    raise e


def extract_features(audio_path, max_pad_len=174, n_mfcc=40):
    try:
        y, sr = librosa.load(audio_path, sr=22050, duration=3)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        if mfcc.shape[1] < max_pad_len:
            mfcc = np.pad(mfcc, ((0, 0), (0, max_pad_len - mfcc.shape[1])), mode='constant')
        else:
            mfcc = mfcc[:, :max_pad_len]
        return mfcc
    except Exception as e:
        print(f" Error extracting features: {e}")
        return None


def predict_emotion(audio_path):
    print(f"\n Analyzing: {audio_path}")
    print("=" * 60)
    
    mfcc = extract_features(audio_path)
    if mfcc is None:
        print(" Failed to extract features.")
        return None
    
    mfcc = mfcc[np.newaxis, ..., np.newaxis]
    prediction = model.predict(mfcc, verbose=0)
    
    predicted_index = np.argmax(prediction)
    predicted_emotion = labels[predicted_index]
    confidence = prediction[0][predicted_index] * 100
    
    print(f"\n Predicted Emotion: {predicted_emotion.upper()}")
    print(f" Confidence: {confidence:.2f}%")
    
    print(f"\n All Predictions:")
    print("-" * 40)
    
    sorted_indices = np.argsort(prediction[0])[::-1]
    for idx in sorted_indices:
        emotion = labels[idx]
        prob = prediction[0][idx] * 100
        bar = "█" * int(prob / 5)
        print(f"   {emotion:12} : {prob:6.2f}% {bar}")
    print("=" * 60)
    return predicted_emotion, confidence


try:
    if 'upload' in globals() and len(upload.value) > 0:
        uploaded_file = list(upload.value)[0]
        filename = uploaded_file.name if hasattr(uploaded_file, 'name') else 'uploaded_audio.wav'
        content = uploaded_file.content if hasattr(uploaded_file, 'content') else upload.value[0]['content']
        
        audio_path = filename 
        with open(audio_path, 'wb') as f:
            f.write(content)
        print(f"\n File saved locally as: {audio_path}")
        
        print("\n Playing audio:")
        try:
            display(Audio(audio_path))
        except:
            pass
        
        predict_emotion(audio_path)
    else:
        print("\n Please upload a file first!")
except NameError:
    print("\n 'upload' widget is not defined.")
except Exception as e:
    print(f" An error occurred: {e}")

 Loading model...
 Model loaded successfully!
 Emotions: ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad']

 File saved locally as: WhatsApp Audio 2025-12-06 at 19.42.03_4c76a56f.mp3

 Playing audio:



 Analyzing: WhatsApp Audio 2025-12-06 at 19.42.03_4c76a56f.mp3

 Predicted Emotion: FEAR
 Confidence: 54.41%

 All Predictions:
----------------------------------------
   Fear         :  54.41% ██████████
   Disgust      :  28.21% █████
   Sad          :  17.30% ███
   Angry        :   0.04% 
   Happy        :   0.03% 
   Neutral      :   0.00% 
